In [1]:
import torch
import torch.onnx
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import util

In [2]:
from session import *
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *

In [3]:
num_classes = 8

In [10]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, num_classes),
    nn.Sigmoid()
)

In [11]:
criterion = nn.BCELoss()
optim_fn = optim.Adam
sess = Session(model_ft, criterion, optim_fn, 1e-2)

In [12]:
sess.load('TrainLayer1')

In [13]:
imsize = 224
dummy_input = Variable(util.to_gpu(torch.randn(1, 3, 224, 224)))

In [14]:
input_names = ["image"]
output_names = ["classification"]

In [15]:
with EvalModel(sess.model):
    torch_out = torch.onnx._export(sess.model,             # model being run
                               dummy_input,                       # model input (or a tuple for multiple inputs)
                               "roaddamage.onnx", # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file
